In [1]:
import pickle
import pandas as pd

In [2]:
def load_in_participant_data(path):
    with open(path, 'rb') as pickle_file:
        loaded_data = pickle.load(pickle_file)

    NeedToSearch = loaded_data['NeedToSearch']
    CorrectSearch = loaded_data['CorrectSearch']
    IncorrectSearch = loaded_data['IncorrectSearch']

    return NeedToSearch, CorrectSearch, IncorrectSearch

In [3]:
path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\Oscillatory_Features_concat\participant_001_Features.pkl"

In [4]:
NeedToSearch, CorrectSearch, IncorrectSearch = load_in_participant_data(path)

array([1.90867817e-11, 2.72678295e-11, 3.10122836e-11, ...,
       8.31097944e-02, 2.76566653e-01, 2.66461701e-01])

In [34]:

from math import floor
import numpy as np

def create_x_y(NeedToSearch, CorrectSearch, IncorrectSearch):
    # Assuming you have three arrays with data
    need_to_search = NeedToSearch[0]
    correct_search = CorrectSearch[0]
    incorrect_search = IncorrectSearch[0]

    correct_search = np.array(correct_search)
    incorrect_search = np.array(incorrect_search)

    correct_search = correct_search[np.random.choice(correct_search.shape[0], floor(len(need_to_search)/2), replace=False)]
    incorrect_search = incorrect_search[np.random.choice(incorrect_search.shape[0], floor(len(need_to_search)/2), replace=False)]

    # Combine the data and create corresponding labels
    X = np.concatenate([need_to_search, correct_search, incorrect_search], axis=0)

    # Create labels (0 for Need to Search, 1 for others)
    y_need_to_search = np.zeros(len(need_to_search))
    y_others = np.ones(len(correct_search) + len(incorrect_search))

    # Concatenate labels and shuffle the data
    y = np.concatenate([y_need_to_search, y_others])
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    X_shuffled = X[indices]
    y_shuffled = y[indices]

    return X_shuffled, y_shuffled



In [37]:
ParticipantList = ["01", "02", "03", "04", "05", "08", "09", "10"]

In [40]:
X_list = []
Y_list = []

for participant in ParticipantList:
    path = fr"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\Oscillatory_Features_concat\participant_0{participant}_Features.pkl"

    NeedToSearch, CorrectSearch, IncorrectSearch = load_in_participant_data(path)

    X, Y = create_x_y(NeedToSearch, CorrectSearch, IncorrectSearch)
    X_list.append(X)
    Y_list.append(Y)






In [43]:
X_all = np.concatenate(X_list, axis=0)
Y_all = np.concatenate(Y_list, axis=0)

In [45]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def get_metrics(model, X_Test, Y_Test):
    y_pred = model.predict(X_Test)
    precision, recall, f1, support = precision_recall_fscore_support(Y_Test, y_pred, average='weighted')
    accuracy = accuracy_score(Y_Test, y_pred)
    print(precision, recall, f1, accuracy)
    metrics = [precision, recall, f1, accuracy]
    return accuracy

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X_all, Y_all, test_size=0.2, random_state=1)

In [58]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [ ]:
metrics = get_metrics(clf, X_test, y_test)

print(metrics)